## Big Data

Big Data se refiere a las técnicas y herramientas utilizadas para procesar grandes volúmenes de datos que no pueden ser manejados por una sola computadora debido a su volumen, velocidad, variedad y veracidad (las 4 "V").

- **Volumen:** Cantidad masiva de datos.

- **Velocidad:** Rapidez con la que se generan y procesan los datos.

- **Variedad:** Diferentes tipos de datos (estructurados, semi-estructurados, no estructurados).

- **Veracidad:** Calidad y confiabilidad de los datos.

## Ecosistema de Apache Hadoop

Apache Hadoop es un framework de código abierto diseñado para el procesamiento distribuido de grandes volúmenes de datos en clústeres de computadoras. Hadoop se compone de varios módulos, entre los que destacan:

- **HDFS (Hadoop Distributed File System):** Sistema de archivos distribuido que almacena datos en múltiples nodos.

- **MapReduce:** Modelo de programación para procesar grandes conjuntos de datos en paralelo.

- **YARN (Yet Another Resource Negotiator):** Gestor de recursos que administra los recursos del clúster y planifica las tareas.

<img src="images\Ecosistena_Apache_Hadoop.png" alt="Ecosistema Apache Hadoop" width="600">

#### 4. **Apache Hive**

Apache Hive es una herramienta construida sobre Hadoop que permite realizar consultas SQL sobre grandes volúmenes de datos almacenados en HDFS. Hive convierte las consultas SQL en trabajos de MapReduce, lo que facilita el análisis de datos para usuarios familiarizados con SQL.

**Integración de Hive con Spark**

Spark puede leer y escribir datos en tablas de Hive, lo que permite combinar la potencia de Spark con la facilidad de uso de Hive para consultas SQL.

### Data Lakehouse
Un Data Lakehouse es un repositorio centralizado que almacena grandes cantidades de datos en su formato crudo, tanto estructurados como no estructurados. Combina las ventajas de un Data Lake (almacenamiento de datos en bruto) con las de un Data Warehouse (gestión de datos estructurados).

## RDD (Resilient Distributed Datasets) 

### **a) Fundamentos de Spark**  

- Los **RDD** (Resilient Distributed Datasets) son la estructura de datos fundamental en Spark. Representan una colección distribuida de elementos que pueden procesarse en paralelo.  
- **Spark SQL** y **DataFrames** están construidos sobre los RDD, pero abstraen gran parte de la complejidad para facilitar el trabajo con datos estructurados.  

---

### **b) Entendimiento de la arquitectura**  

Los **RDD** son la base del procesamiento distribuido en Spark. Son colecciones distribuidas y tolerantes a fallos que se procesan en paralelo en un clúster. Su arquitectura se basa en los siguientes principios:  

#### **1. Distribuido**  
- Los datos se dividen en **particiones** y se distribuyen en los nodos del clúster.  
- Cada partición se procesa en paralelo en diferentes nodos, optimizando el rendimiento.  

#### **2. Resiliente (Tolerante a Fallos)**  
- Los RDD registran su **linaje** (historial de transformaciones) para reconstruir datos en caso de fallos.  
- Si un nodo falla, Spark puede **recrear las particiones perdidas** utilizando el linaje.  

#### **3. Inmutable**  
- Los RDD **no pueden modificarse**. Cada transformación genera un nuevo RDD en lugar de alterar el original.  
- Ejemplos de transformaciones: `map`, `filter`, `reduce`, que producen nuevos RDD.  

#### **4. Lazy Evaluation (Evaluación Perezosa)**  
- Las transformaciones en RDD **no se ejecutan inmediatamente**.  
- Spark espera hasta que se requiere una acción (como `collect` o `count`) para ejecutar las operaciones, optimizando así el procesamiento.  

---

### **c) Transición natural**  
Los **RDD** ofrecen flexibilidad, pero pueden ser menos eficientes para operaciones estructuradas.  
- La transición de **RDD a DataFrames** permite que Spark optimice el procesamiento de datos estructurados mediante un motor de consultas más eficiente.  

---

### **d) Contexto histórico**  
- Spark se diseñó inicialmente con **RDD** como su estructura central.  
- Con el tiempo, se introdujeron **DataFrames y Spark SQL** para simplificar la manipulación de datos estructurados y mejorar el rendimiento.  
- Entender esta evolución es clave para aprovechar al máximo Spark.

**Ejemplo de RDD en Scala:**

In [ ]:
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.SparkSession

// Iniciar una SparkSession
val spark = SparkSession.builder()
  .appName("Introducción a RDD")
  .master("local[*]")
  .getOrCreate()

// Crear un RDD a partir de una lista
val datos: RDD[Int] = spark.sparkContext.parallelize(Seq(1, 2, 3, 4, 5))

// Transformación: Multiplicar cada elemento por 2
val rddTransformado: RDD[Int] = datos.map(x => x * 2)

// Acción: Recopilar los resultados
val resultados: Array[Int] = rddTransformado.collect()

// Mostrar resultados
resultados.foreach(println)

**Ejemplo de transición a DataFrames:**

In [ ]:
// Crear un DataFrame a partir de un RDD
val df = spark.createDataFrame(rddTransformado.map(x => (x, x * 2)), Seq("col1", "col2"))

// Mostrar el DataFrame
df.show()

**Explicación:**
- **Paso 1: Partimos de un RDD**

In [ ]:
rddTransformado.map(x => (x, x * 2))

- Aquí, rddTransformado es un RDD que contiene datos numéricos.
- Se aplica map(x => (x, x * 2)) para transformar cada elemento x en una tupla (x, x * 2), agregando una nueva columna que duplica el valor original.

**Ejemplo de cómo quedaría el RDD después de esta transformación:**

In [ ]:
[(1,2), (2,4), (3,6), (4,8)]

- **Paso 2: Convertimos el RDD en un DataFrame**

In [ ]:
val df = spark.createDataFrame(rddTransformado.map(x => (x, x * 2)), Seq("col1", "col2"))

- Se usa spark.createDataFrame(...) para convertir el RDD en un DataFrame.
- La función recibe el RDD transformado y una lista con los nombres de las columnas: "col1" y "col2".
- El DataFrame resultante tiene una estructura tabular, más parecida a una tabla SQL o un archivo CSV.

- **Paso 3: Visualizamos el DataFrame**

In [ ]:
df.show()

+----+----+
|col1|col2|
+----+----+
|   1|   2|
|   2|   4|
|   3|   6|
|   4|   8|
+----+----+

**Ejemplo de comparación RDD vs DataFrame:**
- **Filtrado en RDD:**

In [ ]:
val rdd = spark.sparkContext.parallelize(Seq(1, 2, 3, 4, 5))
val rddFiltrado = rdd.filter(x => x > 3)
rddFiltrado.collect().foreach(println)

- **Filtrado en DataFrame:**

In [ ]:
val df = spark.createDataFrame(Seq((1), (2), (3), (4), (5))).toDF("valor")
val dfFiltrado = df.filter("valor > 3")
dfFiltrado.show()

## **Apache Spark**

#### **1. ¿Qué es Apache Spark?**

Apache Spark es un motor de procesamiento de datos distribuido y en memoria, diseñado para ser rápido y eficiente en el manejo de grandes volúmenes de datos. A diferencia de Hadoop MapReduce, que escribe y lee datos del disco en cada paso, Spark mantiene los datos en memoria, lo que lo hace hasta 100 veces más rápido para ciertas cargas de trabajo.

#### **Ventajas de Spark sobre Hadoop MapReduce:**

- **Velocidad:** Al procesar datos en memoria, Spark evita la sobrecarga de E/S (entrada/salida) en disco, lo que lo hace ideal para aplicaciones que requieren baja latencia.

- **Facilidad de uso:** Spark ofrece APIs en Scala, Java, Python y R, lo que lo hace accesible para una amplia gama de desarrolladores.

- **Unificación:** Spark integra múltiples componentes (SQL, streaming, machine learning, procesamiento de grafos) en una sola plataforma, eliminando la necesidad de usar herramientas separadas.

- **Tolerancia a fallos:** Spark utiliza RDD (Resilient Distributed Datasets), que son colecciones de datos distribuidos y tolerantes a fallos.

#### **Casos de uso en ciencia de datos:**

- **Procesamiento de datos a gran escala:** Limpieza, transformación y análisis de grandes volúmenes de datos.

- **Machine Learning:** Entrenamiento de modelos con grandes conjuntos de datos usando MLlib.

- **Streaming en tiempo real:** Procesamiento de flujos de datos en tiempo real con Spark Streaming.

- **Análisis de grafos:** Procesamiento de datos en forma de grafos con GraphX.

#### **2. Componentes principales de Spark**

Spark es una plataforma modular que incluye varios componentes para diferentes tipos de procesamiento de datos:

**1. Spark Core:**

- Es el núcleo de Spark y proporciona la funcionalidad básica para la ejecución distribuida de tareas.

- Incluye soporte para RDD, que son la base de todas las operaciones en Spark.

**2. Spark SQL:**

- Permite trabajar con datos estructurados usando consultas SQL.

- Proporciona una interfaz unificada para trabajar con DataFrames y Datasets.

- Ejemplo: Consultas SQL sobre grandes volúmenes de datos almacenados en HDFS o bases de datos.

**3. Spark Streaming:**

- Permite procesar flujos de datos en tiempo real.

- Ejemplo: Análisis de logs en tiempo real o procesamiento de datos de sensores.

**4. MLlib (Machine Learning Library):**

- Biblioteca de machine learning escalable que incluye algoritmos como clasificación, regresión, clustering y recomendación.

- Ejemplo: Entrenamiento de modelos predictivos con grandes conjuntos de datos.

**5. GraphX:**

- Biblioteca para el procesamiento de grafos y análisis de redes.

- Ejemplo: Análisis de redes sociales o detección de comunidades en grafos.

#### **3. SparkSession: Punto de entrada a la funcionalidad de Spark**

SparkSession es la interfaz principal para interactuar con Spark. Es el punto de entrada para crear DataFrames, ejecutar consultas SQL y acceder a otras funcionalidades de Spark.

**¿Qué hace SparkSession?**

- Gestiona la conexión con el clúster de Spark.

- Proporciona acceso a las APIs de Spark (DataFrames, SQL, streaming, etc.).

- Configura las propiedades de la aplicación (por ejemplo, el nombre de la aplicación, el modo de ejecución, etc.).

**Cómo crear una SparkSession en Scala:**

In [ ]:
import org.apache.spark.sql.SparkSession

// Crear una SparkSession
val spark = SparkSession.builder()
  .appName("MiPrimeraAplicacionSpark")  // Nombre de la aplicación
  .master("local[*]")                   // Ejecutar en local con todos los núcleos
  .getOrCreate()

// Ejemplo: Crear un DataFrame a partir de una lista
val datos = Seq(("Alice", 34), ("Bob", 45), ("Cathy", 29))
val df = spark.createDataFrame(datos).toDF("Nombre", "Edad")

// Mostrar el DataFrame
df.show()

**Explicación del código:**  

- **appName("MiPrimeraAplicacionSpark")**: Asigna un nombre a la aplicación, que aparecerá en la interfaz web de Spark.

- **master("local[*]")**: Especifica que la aplicación se ejecutará en modo local, utilizando todos los núcleos disponibles.

- **.getOrCreate()**: Crea una nueva SparkSession o reutiliza una existente.
ete.



**El Shell de Spark en Scala**

- La forma más sencilla de interactuar con Spark es a través del shell de Scala, que se inicia con el comando:

In [ ]:
spark-shell

- Este shell inicia una SparkSession automáticamente, permitiendo ejecutar código Scala directamente en un entorno interactivo.